In [2]:
from bs4 import BeautifulSoup

import re
import requests
import pickle

from multiprocessing import Pool

In [3]:
with open('../data/cats_with_277_pages', 'rb') as f:
     categories_and_max_pages = pickle.load(f)

In [4]:
model_link_and_title_pattern = re.compile(r'<a href="(.*)" title="(.*)" class=')

In [5]:
def parse_category_page(category_page):
    response = requests.get(category_page)
    model_urls_and_titles = re.findall('(?i)<a href=\"(https:\/\/www.cgtrader.com\/3d-models\/[a-z]+\/[^>]+)" title="([^"]+)', str(response.content))
    return model_urls_and_titles

In [6]:
def format_url_pagination(url, page):
    return f"{url}?page={page}"

In [7]:
test_urls = [format_url_pagination(x[0], 1) for x in categories_and_max_pages[0:10]]

In [8]:
test_urls

['https://www.cgtrader.com/3d-models/furniture?page=1',
 'https://www.cgtrader.com/3d-models/interior?page=1',
 'https://www.cgtrader.com/3d-models/architectural?page=1',
 'https://www.cgtrader.com/3d-models/character?page=1',
 'https://www.cgtrader.com/3d-models/house?page=1',
 'https://www.cgtrader.com/3d-models/exterior?page=1',
 'https://www.cgtrader.com/3d-models/modern?page=1',
 'https://www.cgtrader.com/3d-models/decoration?page=1',
 'https://www.cgtrader.com/3d-models/chair?page=1',
 'https://www.cgtrader.com/3d-models/car?page=1']

In [10]:
pages_to_scrape = []

for category, page in categories_and_max_pages:
    for i in range(1, page):
        formatted_url = format_url_pagination(category, i)
        pages_to_scrape.append(formatted_url)
        #model_title_tuples_lst.extend(parse_category_page(formatted_url))

In [24]:
model_title_tuples_lst = []

with Pool(8) as p:
    model_title_tuples_lst.append(p.map(parse_category_page, pages_to_scrape))

In [69]:
flattened_model_endpoints = []

for category in model_title_tuples_lst[0]:
    for model_tuple in category:
        if len(model_tuple[0]) > 1:
            flattened_model_endpoints.append(model_tuple[0])

In [70]:
len(flattened_model_endpoints)

354652

In [72]:
unique_flat_endpoint_urls = list(set(flattened_model_endpoints))

In [82]:
unique_flat_endpoint_urls[187610]

'https://www.cgtrader.com/3d-models/furniture/chair/0582-chair'

In [8]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('/media/void/Seagate Expansion Drive/data/cgtrader2/') if isfile(join('/media/void/Seagate Expansion Drive/data/cgtrader2/', f))]

In [9]:
onlyfiles

['00-3d-decorative-wall-panels-screen-set-0',
 '00021yvonne004',
 '00059karen011-business-woman-sitting-on-the-chair-3d-model',
 '001-hardcover-book',
 '0011-modern-sofa',
 '0014-ottoman',
 '0019-modern-armchair',
 '0021-modern-chair',
 '0023-modern-chair',
 '0025-modern-armchair',
 '0026-modern-sofa',
 '0029-modern-chair',
 '0035-coffee-table',
 '0036-modern-chair',
 '0039-table',
 'ferrari-f500-1952',
 'ferrari-fxxk-evo-2018-red',
 'ferrari-gtc4-lusso-2017-33d3cb12116a371ee563927ade87d461',
 'ferrari-gtc4-lusso-t-2017',
 'ferrari-gtc4lusso',
 'ferrari-gto-1984',
 'ferrari-j50-2017',
 'ferrari-laferrari-2014-c699ab3b-1e84-40d4-9818-b3fdce27e6c7',
 'ferrari-laferrari-9e3ea8a0-1360-474a-9bc9-7173cb9fca83',
 'ferrari-low-poly',
 'ferrari-portofino-ad30f914-92d5-4f76-afa9-4472c4288227',
 'ferrari-roma-2020-mp1',
 'ferrari-testarosa-rigged-and-textured',
 'ferrari-v12-engine-3-liter',
 'ferre-victoria-3d-model',
 'royalton-small-chandelier-brass',
 'royalton-small-pendant-nickel',
 'royalt

In [89]:
endpoints_to_retry = [x[0] for x in list(map(lambda x: (x, x.split('/')[-1]),unique_flat_endpoint_urls)) if x[1] not in onlyfiles]

In [10]:
with open('../data/model_url_endpoints.pkl', 'rb') as f:
    endpoints_to_retry = pickle.load(f)

In [24]:
def save_response_to_disk(endpoint_url):
    try:
        response = requests.get(endpoint_url, 'html')
        filename = endpoint_url.split('/')[-1]
        with open(f'/media/void/Seagate Expansion Drive/data/cgtrader2/{filename}', 'wb') as f:
            pickle.dump(response, f)
    except:
        print(f"could not retrieve file")

In [16]:
model_names = list(map(lambda x: x.split('/')[-1], [x[0] for x in endpoints_to_retry[0]]))

In [19]:
endpoints_to_retry = list(set(model_names).difference(set(onlyfiles)))

In [ ]:
with Pool(8) as p:
    p.map(save_response_to_disk, endpoints_to_retry)

could not retrieve file
could not retrieve file
could not retrieve file


In [29]:
with open('../data/model_url_endpoints.pkl', 'wb') as f:
    pickle.dump(model_title_tuples_lst[0], f)

In [33]:
len(model_title_tuples_lst[0][0]) * len(model_title_tuples_lst[0])

511980

In [7]:
category, page = categories_and_max_pages[0]

In [10]:
formatted_endpoint_url = format_url_pagination(category, 1)